In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Conv1D, Flatten, Dropout, Reshape
from tensorflow.keras.optimizers import Adam

# Load dataset
data = pd.read_csv("Insurance-MEPS.csv")

# Preliminary data preprocessing
numeric_features = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = data.select_dtypes(include=['object']).columns.tolist()

# Removing the target variable from numeric features if it's included
if 'TOTSLF21' in numeric_features:
    numeric_features.remove('TOTSLF21')

# Define transformations for numeric columns (impute missing values and scale)
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

# Define transformations for categorical data (impute and encode)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Create a preprocessor object to aggregate transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Split the dataset into training and test sets
X = data.drop('TOTSLF21', axis=1)
y = data['TOTSLF21']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply preprocessing
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Reshape the input to make it suitable for Conv1D (adding a channel dimension)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Define the model
input_layer = Input(shape=(X_train.shape[1], 1))
conv1d_layer = Conv1D(64, kernel_size=3, activation='relu')(input_layer)
flatten_layer = Flatten()(conv1d_layer)
dense_layer = Dense(64, activation='relu')(flatten_layer)
dropout_layer = Dropout(0.5)(dense_layer)
output_layer = Dense(1)(dropout_layer)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model
y_pred = model.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred)
print(f"Test MSE: {test_mse}")


Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 104ms/step - loss: 9261914.0000 - val_loss: 2970282.0000
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 106ms/step - loss: 7652624.5000 - val_loss: 2809884.0000
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 111ms/step - loss: 5478140.5000 - val_loss: 2731216.7500
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 114ms/step - loss: 4120866.5000 - val_loss: 2654462.0000
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 111ms/step - loss: 6730373.5000 - val_loss: 2599482.0000
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 112ms/step - loss: 2935017.7500 - val_loss: 2534055.7500
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 113ms/step - loss: 4097069.7500 - val_loss: 2514284.0000
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 113ms/step - loss: 9141567.0000 - val_loss: 2456175.2500
Epoch 9/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 122ms/step - loss: 8687530.0000 - val_loss: 2409066.5000
Epoch 10/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 135ms/step - loss: 7506217.0000 - val_loss: 2364906.7500
Epoch 11/50
20/20 ━